In [82]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt


In [83]:
df = pd.read_csv('insurance.csv')

In [ ]:
model = smf.ols(formula='charges ~ bmi + sex + age + smoker + children', data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.750
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     798.0
Date:                Sun, 23 Jun 2024   Prob (F-statistic):               0.00
Time:                        12:48:47   Log-Likelihood:                -13551.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1332   BIC:                         2.715e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -1.205e+04    951.260    -12.670      0.000   -1.39e+04   -1.02e+04
sex[T.male]    -128.6399    333.361     -0.386      0.700    -782.609     525.329
smoker[T.yes]  2.382e+04    412.523     57.750      0.000     2.3e+04    2.46e+04
bmi             322.3642     27.419     11.757      0.000     268.576     376.153
age             257.7350     11.904     21.651      0.000     234.383     281.087
children        474.4111    137.856      3.441      0.001     203.973     744.849
==============================================================================
Omnibus:                      301.103   Durbin-Watson:                   2.087
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              720.380
Skew:                           1.214   Prob(JB):                    3.73e-157
Kurtosis:                       5.650   Cond. No.                         295.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""